# Test 1

### Packages

In [ ]:
import pandas as pd
from datetime import timedelta  
import os

os.chdir('/Users/tristanmayer/Documents/GitHub/taxi-time')
from src.preprocessing import *
from src.master import *

### Data

In [ ]:
df1 = cleaning_airport_df('/Users/tristanmayer/Desktop/Supercase Eleven/data/0. Airport data/Airport_Data.csv')

In [ ]:
df2 = get_df_of_obs1(df1)

In [ ]:
df3 = get_target_values(df2)

In [ ]:
df = master_preprocessing_X('/Users/tristanmayer/Desktop/Supercase Eleven/data/0. Airport data/Airport_Data.csv',
                            '/Users/tristanmayer/Desktop/Supercase Eleven/data/2. Weather data/weather_data_prep.csv',
                            '/Users/tristanmayer/Desktop/Supercase Eleven/data/1. AC characteristics/ACchar.xlsx',
                            '/Users/tristanmayer/Documents/GitHub/taxi-time/Correspondance.pkl')

In [ ]:
# number of planes in the airport at time_t (a date in string format)
# number of planes in block at time_t (a date in string format)
#### need to reconcile flights with plane (we only know details about flight so not the same line for arrival and departure from the stand -> see the dates regarding one stand)

# number of planes in movement at time_t (a date in string format)
## input: date as a string
## output: integer
def get_nb_of_planes_in_movement(time_t):
    t = pd.to_datetime(time_t)
    res = len(df1[((df1['aldt']<t) & (df1['aibt']>t))|
                  ((df1['aobt']<t) & (df1['atot']>t))])
    return res

# number of planes that have landed on the runway_R in the last M_min at time_t
## input: runway as a string; date as a string; timedelta as an integer (nb of minutes)
## output: integer
def get_nb_of_planes_runway_in_last_M_min(runway_R, time_t, M_min):
    t1 = pd.to_datetime(time_t)
    t2 = t1-timedelta(minutes = M_min)
    res = len(df1[(df1['runway']==runway_R) &
                  (df1['aldt']<t1) &
                  (df1['aldt']>t2)])
    return res


# number of planes that have stayed at the stand_S in the last M min at time_t
def get_nb_of_planes_stand_in_last_M_min(stand_S, time_t, M_min):
    t1 = pd.to_datetime(time_t)
    t2 = t1-timedelta(minutes = M_min)
    res = len(df1[(df1['stand']==stand_S) &
                  (df1['aibt']<t1) &
                  (df1['aibt']>t2)])
    return res

# average taxi-time of the planes that have landed on runway_R in the last M min at time_t
def average_taxitime_runway_last_X_min(runway_R, time_t, M_min): 
    t1 = pd.to_datetime(time_t)
    t2 = t1-timedelta(minutes = M_min)
    list_of_index = df1[(df1['runway']==runway_R) & 
                        (df1['aibt']<t1) & 
                        (df1['aibt']>t2)].index
    res = df3.loc[list_of_index][0].mean()
    return res

# average taxi-time of the planes that stayed at stand_S in the last M min at time_t
def average_taxitime_stand_last_X_min(stand_S, time_t, M_min): 
    t1 = pd.to_datetime(time_t)
    t2 = t1-timedelta(minutes = M_min)
    list_of_index = df1[(df1['stand']==stand_S) & 
                        (df1['aibt']<t1) & 
                        (df1['aibt']>t2)].index
    res = df3.loc[list_of_index][0].mean()
    return res

# taxi_time of the last plane that have landed on runway_R at time_t
def get_last_taxitime_runway(runway_R, time_t):
    t1 = pd.to_datetime(time_t)
    res = df3.loc[df1[(df1['runway']==runway_R) & (df1['aibt']<t1)]['aibt'].idxmax()][0]
    return res

# taxi_time of the last plane that have stayed at stand_S at time_t
def get_last_taxitime_stand(stand_S, time_t):
    t1 = pd.to_datetime(time_t)
    res = df3.loc[df1[(df1['stand']==stand_S) & (df1['aibt']<t1)]['aibt'].idxmax()][0]
    return res

    
# taxi_time of the last plane that have gone from runway_R to stand_S at time_t
def get_last_taxitime_path(runway_R, stand_S, time_t):
    t1 = pd.to_datetime(time_t)
    res = df3.loc[df1[(df1['runway']==runway_R) &
                       (df1['stand']==stand_S) & 
                       (df1['aibt']<t1)]['aibt'].idxmax()][0]
    return res

In [ ]:
time_t = '2018-08-01 19:59:00'

In [ ]:
df1.head()

In [ ]:
t = pd.to_datetime(time_t)
len(df1[((df1['aldt']<t) & (df1['aibt']>t))|
        ((df1['aobt']<t) & (df1['atot']>t))])

In [ ]:
t1 = pd.to_datetime(time_t)
t2 = t1-timedelta(minutes = 300)
len(df1[(df1['runway']=='RUNWAY01')&
        (df1['aldt']<t1) &
        (df1['aldt']>t2)])

In [ ]:
t1 = pd.to_datetime(time_t)
t2 = t1-timedelta(minutes = 300)
res = len(df1[(df1['stand']=='STAND03') &
                  (df1['aibt']<t1) &
                  (df1['aibt']>t2)])
res

In [ ]:
t1 = pd.to_datetime(time_t)
t2 = t1-timedelta(minutes = 30000)
list_of_index = df1[(df1['stand']=='STAND03') & 
                    (df1['aibt']<t1) & 
                    (df1['aibt']>t2)].index
res = df3.loc[list_of_index][0].mean()
res

In [ ]:
t1 = pd.to_datetime(time_t)
df3.iloc[df1[(df1['runway']=='RUNWAY01') & (df1['aibt']<t1)]['aibt'].idxmax()][0]
         
         
#.sort_values('aibt', ascending=False).iloc[0].name][0]

In [ ]:
df1[(df1['stand']=='STAND03')]

In [ ]:
df1[(df1['runway']=='RUNWAY02') & (df1['aibt']<pd.to_datetime('2018-10-12 13:18:00'))]['aibt'].idxmax()

In [ ]:
df3.loc[141628]

## Test des fonctions

In [ ]:
get_nb_of_planes_in_movement('2018-10-12 13:18:00')

In [ ]:
get_nb_of_planes_runway_in_last_M_min('RUNWAY02','2018-10-12 13:18:00', 2000)

In [ ]:
get_nb_of_planes_stand_in_last_M_min('STAND104', '2018-10-12 13:18:00', 2000)

In [ ]:
average_taxitime_runway_last_X_min('RUNWAY02','2018-10-12 13:18:00', 2000)

In [ ]:
average_taxitime_stand_last_X_min('STAND104', '2018-10-12 13:18:00', 2000)

In [ ]:
get_last_taxitime_runway('RUNWAY02','2018-10-12 13:18:00')

In [ ]:
get_last_taxitime_stand('STAND104','2018-10-12 13:18:00')

In [ ]:
get_last_taxitime_path('RUNWAY02', 'STAND104', '2018-10-12 13:18:00')

#### deal with this pb: sometimes first time a plane takes this path

In [ ]:
#df3.loc[
df1[(df1['runway']=='RUNWAY02') &
    (df1['stand']=='STAND104')]# &
   # (df1['aibt']<pd.to_datetime('2018-10-12 13:18:00'))]#['aibt']#.idxmax()][0]

In [ ]:
df1['aibt'].min()

In [ ]:
df1['aibt'].max()

In [ ]:
for ind, row in df[:100][['runway', 'aldt']].iterrows():
    sys.stdout.write("\r{0}".format((float(ind)/len(df[:100]))*100))
    sys.stdout.flush()

In [ ]:
df1[['runway', 'aldt']].dropna()

In [ ]:
df.y.rolling(window=20).mean()

In [ ]:
df1[['runway', 'aldt']].dropna()

In [ ]:
df_test = df1[['runway', 'aldt']].dropna()

In [ ]:
df_test = df2.join(df3)
df_test[df_test['runway']=='RUNWAY01'][['aldt', 0]].rename(columns={'aldt':'ds',0:'y'})\
.sort_values('ds').y.rolling(window=10).mean()

In [ ]:
df2.join(df3)[['aldt', 0]].rename(columns={'aldt':'ds',0:'y'})\
.sort_values('ds').y.rolling(window=10).mean()

In [ ]:
df['aldt'].apply(lambda x : ((df['aldt']<x)&(df['aldt']>x-timedelta(minutes = 30)).sum()))

In [ ]:
df['rolling average'] = df2.join(df3)[['aldt', 0]].rename(columns={'aldt':'ds',0:'y'})\
                            .sort_values('ds').y.rolling(window=10).mean().values

In [ ]:
df

### TEST 2

In [1]:
import pandas as pd
from datetime import timedelta  
import os

os.chdir('/Users/tristanmayer/Documents/GitHub/taxi-time')
from src.preprocessing import *
from src.master import *

/Users/tristanmayer/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df_train = master_preprocessing_X('/Users/tristanmayer/Desktop/Supercase Eleven/data/0. Airport data/Airport_Data.csv',
                            '/Users/tristanmayer/Desktop/Supercase Eleven/data/2. Weather data/weather_data_prep.csv',
                            '/Users/tristanmayer/Desktop/Supercase Eleven/data/1. AC characteristics/ACchar.xlsx',
                            '/Users/tristanmayer/Documents/GitHub/taxi-time/Correspondance.pkl')

/Users/tristanmayer/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_test = master_preprocessing_X('/Users/tristanmayer/Desktop/Supercase Eleven/data/Test_Set_Airport_Data.csv',
                            '/Users/tristanmayer/Desktop/Supercase Eleven/data/2. Weather data/weather_data_prep.csv',
                            '/Users/tristanmayer/Desktop/Supercase Eleven/data/1. AC characteristics/ACchar.xlsx',
                            '/Users/tristanmayer/Documents/GitHub/taxi-time/Correspondance.pkl')

In [4]:
df_train['TESTING'] = 0
df_test['TESTING'] = 1

In [5]:
df_tot = pd.concat([df_train, df_test])

In [19]:
df_tot = df_tot.reset_index(drop=True)

In [20]:
df_tot

,carrier,flight,acType,sto,runway,stand,aldt,eibt,cibt,aibt,...,MTOW,Max Ramp Max Taxi,Main Gear Config,ICAO Code,Wake Category,ATCT Weight Class,Years Manufactured,Note,"Parking Area (WS x Length), sf",TESTING
0,CAR1,2519,b737/9-wl,2018-07-30 04:45:00,RUNWAY02,STAND04,2018-07-30 04:25:00,2018-07-30 04:33:00,2018-07-30 04:33:00,7/30/2018 4:29,...,174200,174700,D,B739,M,Large Jet Eqpt,tbd,tbd,16223.1,0
1,CAR1,1757,b757/2-wl,2018-07-30 08:53:00,RUNWAY02,STAND07,2018-07-30 08:14:00,2018-07-30 08:19:00,2018-07-30 08:18:00,7/30/2018 8:18,...,255500,256000,2D,B752,M,B757 Eqpt,tbd,tbd,20919.9,0
2,CAR1,1636,a321/2,2018-07-30 08:49:00,RUNWAY01,STAND08,2018-07-30 08:26:00,2018-07-30 08:31:00,2018-07-30 08:32:00,7/30/2018 8:29,...,206132,207014,D,A321,M,Large Jet Eqpt,tbd,tbd,16337.8,0
3,CAR1,2048,b737/9-wl,2018-07-30 08:56:00,RUNWAY02,STAND09,2018-07-30 08:42:00,2018-07-30 08:47:00,2018-07-30 08:45:00,7/30/2018 8:46,...,174200,174700,D,B739,M,Large Jet Eqpt,tbd,tbd,16223.1,0
4,CAR1,2510,a321/2,2018-07-30 09:14:00,RUNWAY02,STAND11,2018-07-30 08:52:00,2018-07-30 08:57:00,2018-07-30 08:57:00,7/30/2018 8:57,...,206132,207014,D,A321,M,Large Jet Eqpt,tbd,tbd,16337.8,0
5,CAR1,110,a330/3,2018-07-30 09:30:00,RUNWAY01,STAND14,2018-07-30 09:07:00,2018-07-30 09:15:00,2018-07-30 09:16:00,7/30/2018 9:17,...,533519,535503,2D,A333,H,Heavy Eqpt,tbd,tbd,41344.5,0
6,CAR1,423,b737/9-wl,2018-07-30 09:23:00,RUNWAY02,STAND15,2018-07-30 09:10:00,2018-07-30 09:16:00,2018-07-30 09:17:00,7/30/2018 9:16,...,174200,174700,D,B739,M,Large Jet Eqpt,tbd,tbd,16223.1,0
7,CAR1,146,b767/3-wl,2018-07-30 09:32:00,RUNWAY01,STAND16,2018-07-30 09:10:00,2018-07-30 09:18:00,2018-07-30 09:15:00,7/30/2018 9:17,...,412000,413000,2D,B763,H,Heavy Eqpt,tbd,tbd,30101.8,0
8,CAR1,1943,b777/2-lr,2018-07-30 08:14:00,RUNWAY01,STAND03,2018-07-30 09:21:00,2018-07-30 09:28:00,2018-07-30 09:27:00,7/30/2018 9:28,...,766000,768000,3D,B772,H,Heavy Eqpt,tbd,tbd,44447.6,0
9,CAR1,2678,b737/9-wl,2018-07-30 09:47:00,RUNWAY03,STAND17,2018-07-30 09:27:00,2018-07-30 09:35:00,2018-07-30 09:33:00,7/30/2018 9:33,...,174200,174700,D,B739,M,Large Jet Eqpt,tbd,tbd,16223.1,0


In [21]:
df_target = create_target(df_tot)

In [22]:
col_numerical = ['AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN','WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT02', 'WT03', 'WT08',
                'nb of planes in movement in the plane',
                'rolling average same runway',
                 'rolling average same stand',
                 'rolling average same runway & same stand', 'TESTING']
col_dummies = ['SnowProxi']
col_to_drop=[ 'flight','sto', 'aldt', 'eibt','aibt',
       'cibt', 'chocks_on', 'Manufacturer','Physical Class (Engine)',
       '# Engines',
       'Wingtip Configuration',
       'Tail Height, ft\n(@ OEW)', 'Wheelbase, ft',
       'Cockpit to Main Gear (CMG)', 'MGW\n(Outer to Outer)',
       'Max Ramp\nMax Taxi', 'Main Gear Config', 'ICAO Code', 'Years Manufactured', 'Note',
       'Parking Area (WS x Length), sf','Model','MTOW','Wingspan, ft', 'Length, ft','Approach Speed\n(Vref)',
       'target']
col_to_target_encode=['stand','carrier', 'acType','runway','ATCT Weight Class','Wake Category','AAC', 'ADG', 'TDG']

In [23]:
df_tot1 = features_pimpage(df_tot,col_numerical,col_to_drop,col_dummies, col_to_target_encode,df_target,
                           path_feature ="Feature_congestion_movement_tot.pkl" ,CatBoost=False)

In [31]:
X_train = df_tot1[0][df_tot1[0]['TESTING']==0]

In [32]:
df_tot1[1].loc[X_train.index]

,target
606,720.0
654,780.0
663,480.0
664,540.0
665,420.0
684,780.0
709,840.0
725,420.0
774,720.0
777,420.0
